In [3]:
import cv2 # OpenCV
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import pandas as pd

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path = "D:\FULL"

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   validation_split=0.5)
training_set = train_datagen.flow_from_directory(path+"/train",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset='training')

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)
validation_set = train_datagen.flow_from_directory(path+"/test",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,

                                                class_mode='categorical',
                                                subset='training')

Found 15923 images belonging to 7 classes.
Found 3976 images belonging to 7 classes.


In [22]:
resnetv2 = keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
# KALO FULL DIGANTI FALSE 
resnetv2.trainable = False
print(len(resnetv2.layers))

192


In [ ]:
# i = 0
# for layer in inceptionv3.layers:
#   if i < 280:
#     layer.trainable = False
#   i+=1

In [23]:
#Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

input = tf.keras.layers.Input(shape=(224, 224, 3))
# Apply data augmentation to the input
x = data_augmentation(input)
x = resnetv2(input, training=False)
# x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
resnetv2 = tf.keras.models.Model(inputs = input, outputs = x, name = "resnetv2")

resnetv2.summary()

Model: "resnetv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 1000)           │    25,613,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │         7,007 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,620,807 (97.74 MB)

 Trainable params: 7,007 (27.37 KB)

 Non-trainable params: 25,613,800 (97.71 MB)

In [24]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    epochs=5,
                    steps_per_epoch=steps_per_epoch
                    )

Epoch 1/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 232s 4s/step - accuracy: 0.2164 - loss: 1.9369
Epoch 2/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 227s 4s/step - accuracy: 0.2502 - loss: 1.9127
Epoch 3/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 227s 4s/step - accuracy: 0.2447 - loss: 1.8942
Epoch 4/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 224s 4s/step - accuracy: 0.2582 - loss: 1.8728
Epoch 5/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.2500 - loss: 1.8633 


In [25]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    epochs=60,
                    steps_per_epoch=steps_per_epoch
                    )

Epoch 1/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 246s 4s/step - accuracy: 0.2582 - loss: 1.8636
Epoch 2/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 230s 4s/step - accuracy: 0.2491 - loss: 1.8505
Epoch 3/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 230s 4s/step - accuracy: 0.2433 - loss: 1.8413
Epoch 4/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 232s 4s/step - accuracy: 0.2578 - loss: 1.8354
Epoch 5/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.1719 - loss: 1.8269 
Epoch 6/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 233s 4s/step - accuracy: 0.2613 - loss: 1.8277
Epoch 7/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 231s 4s/step - accuracy: 0.2590 - loss: 1.8206
Epoch 8/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 232s 4s/step - accuracy: 0.2682 - loss: 1.8165
Epoch 9/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 233s 4s/step - accuracy: 0.2672 - loss: 1.7953
Epoch 10/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 311us/step - accuracy: 0.2188 - loss: 1.8220
Epoch 11/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 232s 4s/step - accuracy: 0.2502 - loss: 1.8126
Epoch 12/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 232s 4s/step - accuracy

In [26]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    epochs=45,
                    steps_per_epoch=steps_per_epoch
                    )

Epoch 1/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 270s 4s/step - accuracy: 0.2985 - loss: 1.7593
Epoch 2/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 249s 4s/step - accuracy: 0.3182 - loss: 1.7433
Epoch 3/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 249s 4s/step - accuracy: 0.3149 - loss: 1.7349
Epoch 4/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 244s 4s/step - accuracy: 0.3094 - loss: 1.7332
Epoch 5/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 498us/step - accuracy: 0.2812 - loss: 1.6565
Epoch 6/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 227s 4s/step - accuracy: 0.3125 - loss: 1.7296
Epoch 7/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 231s 4s/step - accuracy: 0.3059 - loss: 1.7339
Epoch 8/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 231s 4s/step - accuracy: 0.3230 - loss: 1.7244
Epoch 9/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 231s 4s/step - accuracy: 0.3204 - loss: 1.7267
Epoch 10/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 262us/step - accuracy: 0.2969 - loss: 1.8456
Epoch 11/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 232s 4s/step - accuracy: 0.3150 - loss: 1.7280
Epoch 12/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 231s 4s/step - accurac

In [4]:
new_resnetv2_full = tf.keras.models.load_model("resnetv2-full-110.keras")

c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

new_resnetv2_full.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 50
history = new_resnetv2_full.fit(x=training_set,
                    epochs=60,
                    steps_per_epoch=steps_per_epoch
                    )

Epoch 1/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 276s 4s/step - accuracy: 0.3261 - loss: 1.7153
Epoch 2/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 250s 4s/step - accuracy: 0.3305 - loss: 1.7134
Epoch 3/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 249s 4s/step - accuracy: 0.3072 - loss: 1.7170
Epoch 4/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 247s 4s/step - accuracy: 0.3275 - loss: 1.7189
Epoch 5/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.3594 - loss: 1.7551 
Epoch 6/60


c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 239s 4s/step - accuracy: 0.3252 - loss: 1.7144
Epoch 7/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 235s 4s/step - accuracy: 0.3111 - loss: 1.7255
Epoch 8/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 236s 4s/step - accuracy: 0.3228 - loss: 1.7226
Epoch 9/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 237s 4s/step - accuracy: 0.3343 - loss: 1.7239
Epoch 10/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 413us/step - accuracy: 0.3906 - loss: 1.6532
Epoch 11/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 242s 4s/step - accuracy: 0.3184 - loss: 1.7177
Epoch 12/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 249s 4s/step - accuracy: 0.3374 - loss: 1.6985
Epoch 13/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 241s 4s/step - accuracy: 0.3238 - loss: 1.7103
Epoch 14/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 239s 4s/step - accuracy: 0.3365 - loss: 1.7069
Epoch 15/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 706us/step - accuracy: 0.3906 - loss: 1.6614
Epoch 16/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 234s 4s/step - accuracy: 0.3268 - loss: 1.7140
Epoch 17/60
62/62 ━━━━━━━━━━━━━━━━━━━━ 236s 4s/step - accuracy: 0.3

In [ ]:
new_resnetv2_full.save("resnetv2-full-170.keras")

In [4]:
new_resnetv3_full = tf.keras.models.load_model("resnetv2-full-170.keras")